# Machine Learning - CatBoost

In [15]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
required_libs = [ ("numpy", "numpy"),
                 ("pandas", "pandas"),
                 ("seaborn", "seaborn"),
                 ("matplotlib", "matplotlib"),
                 ("catboost", "catboost"),
                 ("sklearn", "sklearn"),
                 ("ipywidgets", "ipywidgets"),
                 ("shap", "shap"),
                ]

In [7]:
def is_lib_exists(name):
    import importlib
    lib = importlib.util.find_spec(name)
    return lib is not None

In [8]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
Installing catboost
     |████████████████████████████████| 64.8MB 67kB/s 
sklearn exists
ipywidgets exists
Installing shap
     |████████████████████████████████| 276kB 2.7MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394118 sha256=b20214ae73d83331dd07aac5de377fd32dbb3dc402cc1645e2d6b217d302ee60
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [9]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import traceback
import catboost
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
from catboost import *
from catboost import datasets
np.set_printoptions(precision=4)

In [11]:
%matplotlib inline
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [12]:
from IPython.display import display

In [13]:
# check catboost version
print(catboost.__version__)
!python --version

0.23.2
Python 3.6.9


In [22]:
# colab setup
if COLAB:
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

Cloning into 'dl-projects'...
remote: Enumerating objects: 140, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 2117 (delta 85), reused 32 (delta 14), pack-reused 1977
Receiving objects: 100% (2117/2117), 79.28 MiB | 13.30 MiB/s, done.
Resolving deltas: 100% (1310/1310), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
total 4948M
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr.ft
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr_orig.pkl
-rw------- 1 root root   17M Mar  1 05:47 feature_matrix_2020_feb.ft
-rw------- 1 root root   17M Mar  1 05:39 feature_matrix_2020_jan.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul_orig.pkl
-rw------- 1 root root   17M Jul  5 07:30 feature_matrix_2020_jun.ft
-rw------- 1 

## EDA

In [20]:
from preprocess import *

In [21]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [23]:
jan_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jan.ft")
feb_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_feb.ft")
mar_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_mar.ft")
apr_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_apr.ft")
may_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_may.ft")
jun_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jun.ft")
jul_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jul.ft")

In [26]:
# Compare July data
jul_2020_orig = pd.read_pickle(DATASET_PATH/f"feature_matrix_2020_jul_orig.pkl")

In [39]:
# Compare July data
jul_2020.shape, jul_2020_orig.shape
display(jul_2020.columns.difference(jul_2020_orig.columns))
display(jul_2020_orig.columns.difference(jul_2020.columns))

((10000, 217), (10000, 213))

Index(['LAST(Results.PrizeType)_Prize', 'MODE(Results.PrizeType)_Prize',
       'NumberId', 'month', 'time', 'year'],
      dtype='object')

Index(['LAST(Results.PrizeType)', 'MODE(Results.PrizeType)'], dtype='object')

In [63]:
#jul_2020_orig.dtypes.head(10)
jul_2020_orig.shape
display(jul_2020_orig["MODE(Results.PrizeType)"].head(300))
set(jul_2020_orig["MODE(Results.PrizeType)"].values)

(10000, 213)

NumberId  time      
115       2020-07-01    Prize
204       2020-07-01    Prize
318       2020-07-01    Prize
513       2020-07-01    Prize
793       2020-07-01    Prize
                        ...  
9285      2020-07-01    Prize
8726      2020-07-01    Prize
93        2020-07-01    Prize
6128      2020-07-01    Prize
1096      2020-07-01    Prize
Name: MODE(Results.PrizeType), Length: 300, dtype: object

{'Prize'}